<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DBSWIM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# install
!apt install postgresql postgresql-contrib &>log
!service postgresql restart

# PostGRES SQL Settings
!sudo -u postgres psql -c "CREATE USER postgres WITH SUPERUSER
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"
!sudo -u postgres psql -c "DROP TABLE aircraft"

# DATA Locationbs:
basepath='/content/drive/MyDrive/dataswim'

#import Libraries
from xml.dom.minidom import parse, parseString
import xml.etree.ElementTree as ET
import xml.dom.minidom

import os
import psycopg2 as ps
import pandas as pd

DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASS = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

def tag_real(label):
    DEP_tmp=elem.tag.find(label)
    if DEP_tmp>0:
        #print(elem.tag)
        DEP=elem.tag[DEP_tmp:len(elem.tag)]
        #print(elem.tag[DEP_tmp:len(elem.tag)])
        #print('%s: %s'%(DEP,subelem.text))
        realvalue=subelem.text
    else:
        realvalue=''

    return realvalue


def tag_real2(label):
    DEP_tmp=subelem.tag.find(label)
    if DEP_tmp>0:
        DEP=subelem.tag[DEP_tmp:len(subelem.tag)]
        #print('%s: %s'%(DEP,subelem.text))
        realvalue=subelem.text
    else:
        realvalue=''

    return realvalue
    #return DEP_tmp

def tag_real3(label):
    DEP_tmp=elem.tag.find(label)
    if DEP_tmp>0:
        #print(elem.tag)
        DEP=elem.tag[DEP_tmp:len(elem.tag)]
        largo=len(elem.text)
        #print(largo)
        #if len(elem.attrib)>0:
            #print(elem.text)

        #    if largo>0:
        #        print('%s: %s FAA Identifier: %s'%(DEP,elem.attrib,elem.text))
        #    if largo==0:
        #       print('%s: %s'%(DEP,elem.attrib))


            #else:
            #    print('%s: %s'%(DEP,elem.attrib))
    #print()
    return DEP_tmp,elem.attrib

def var_validator(VARIABLE):
     try:
          VARIABLE
     except NameError:
            #print("well, it WASN'T defined after all!")
            VARIABLE='N/A'
     #else:
           #print("sure, %s was defined."%VARIABLE)
     return VARIABLE


def tag_real4(label):
    DEP_tmp=elem.tag.find(label)
    if DEP_tmp>0:
        DEP=elem.tag[DEP_tmp:len(elem.tag)]
        largo=len(elem.text)

    return DEP_tmp,elem.attrib


conn = ps.connect(database=DB_NAME,
							user=DB_USER,
							password=DB_PASS,
							host=DB_HOST,
							port=DB_PORT)
#print("Database connected successfully")

cur = conn.cursor() # creating a cursor
cur.execute("""
                            CREATE TABLE aircraft
                            (
                                ID INT NOT NULL,
                                gufi text NOT NULL,
                                aircraftId TEXT  NOT NULL,
                                igtd TEXT NOT NULL,
                                departurePoint TEXT NOT NULL,
                                arrivalPoint TEXT NOT NULL,
                                route TEXT NOT NULL,
                                Airline TEXT NOT NULL,
                                primary key (aircraftId,igtd,departurePoint,arrivalPoint)
                            )
                            """)

conn.commit()
print("Table Aircraft Created successfully")
conn.close()

for entry in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, entry)):
        filename=os.path.join(basepath, entry)
        print('FILENAME: %s'%filename)

        if int(entry)>10:
            print()
            #print(entry)
            print()
            print()

            tree = ET.parse(filename)
            root = tree.getroot()

            n=0

            for elem in root.iter():
                #print()
                n=n+1
                ID=n
                #print('RECORD: %s'%n)
                if len(elem.attrib)>0:
                    QC=tag_real3('star')
                    QC=tag_real3('fix')
                    QC=tag_real3('waypoint')


                    [QC,values]=tag_real4('fltdMessage')

                    #fltdMessage: {'acid': 'DAL1431', 'airline': 'DAL', 'arrArpt': 'KATL', 'cdmPart': 'false', 'depArpt': 'TJSJ',
                    #              'fdTrigger': 'HCS_AMENDMENT_MSG', 'flightRef': '50227768', 'major': 'DAL', 'msgType': 'flightPlanAmendmentInformation', 'sensitivity': 'A',
                    #                'sourceFacility': 'KZJX', 'sourceTimeStamp': '2023-09-20T22:58:00Z'}

                    if QC>0:
                       flightnumber=values['acid']
                       airline=values['airline']
                       if len(values)>12:
                           departureairport=values['depArpt']
                           try:
                               arrivalairport=values['arrArpt']
                           except:
                               arrivalairport=values['arrArpt']
                           #print('Flight Number: %s, Airline: %s, Airport of Departure: %s, Airport of Arrival: %s'%(flightnumber,airline,departureairport,arrivalairport))
                       #else:
                       #    print('Flight Number; %s, Airline: %s'%(flightnumber,airline))
                       if len(values)==11:
                           try:
                               departureairport=values['depArpt']
                               #print('Flight Number: %s, Airline: %s, Airport of Departure: %s'%(flightnumber,airline,departureairport))
                           except:
                               arrivalairport=values['arrArpt']
                               #print('Flight Number: %s, Airline: %s, Airport of Arrival: %s'%(flightnumber,airline,arrivalairport))
                    [QC,values]=tag_real4('qualifiedAircraftId')
                    if QC>0:
                       aircraftCategory=values['aircraftCategory']
                       userCategory=values['userCategory']

                       #{'aircraftCategory': 'JET', 'userCategory': 'COMMERCIAL'}
                       print('Aircraft Category: %s, User Category: %s'%(aircraftCategory,userCategory))

                for subelem in elem:

                    #print(subelem.tag)
                    if len(elem.attrib)>0:
                        QC=tag_real3('star')
                        QC=tag_real3('fix')
                        QC=tag_real3('waypoint')

                    if subelem.text is None:
                       subelem.text=''

                    if len(subelem.text)>0:

                        DEP_tmp=tag_real('departurePoint')

                        if len(DEP_tmp)>0:
                            DEP=DEP_tmp

                        ARR_tmp=tag_real('arrivalPoint')

                        if len(ARR_tmp)>0:
                            ARR=ARR_tmp

                        #print(elem.tag[0:len(elem.tag)])
                        DEP_tmp=tag_real2('status')
                        #print()

                        EP_tmp=tag_real('aircraftCategory')
                        EP_tmp=tag_real('userCategory')

                        ROUTE_tmp=tag_real2('routeOfFlight')


                        if len(ROUTE_tmp)>0:
                            ROUTE=ROUTE_tmp

                        #QC=tag_real2('waypoint')
                        #QC=tag_real3('star')

                        #print()

                        ACID_tmp=tag_real2('aircraftId')
                        if len(ACID_tmp)>0:
                            print('acid: %s'%ACID_tmp)
                            ACID=ACID_tmp
                            AIRLINE=ACID[0:3]

                        GUFI_tmp=tag_real2('gufi')
                        if len(GUFI_tmp)>0:
                            GUFI=GUFI_tmp

                        SCHE_tmp=tag_real2('igtd')

                        if len(SCHE_tmp)>0:
                            SCHE=SCHE_tmp

                        EP_tmp=tag_real2('bentryTm')
                        EP_tmp=tag_real2('createTm')
                        EP_tmp=tag_real2('eentryTm')
                        EP_tmp=tag_real2('exitTm')
                        EP_tmp=tag_real2('extendExitTm')
                        EP_tmp=tag_real2('ientryTm')
                        EP_tmp=tag_real2('oentryTm')
                        EP_tmp=tag_real2('entryLat')
                        EP_tmp=tag_real2('entryLon')

                        EP_tmp=tag_real2('entryHeading')
                        EP_tmp=tag_real2('exitInd')
                        EP_tmp=tag_real2('fcaId')
                        EP_tmp=tag_real2('fcaName')
                        EP_tmp=tag_real2('lastUpdate')

                        EP_tmp=tag_real2('flightStatus')
                        EP_tmp=tag_real2('aircraftModel')

                        conn = ps.connect(database=DB_NAME,
                                user=DB_USER,
                                password=DB_PASS,
                                host=DB_HOST,
                                port=DB_PORT)
                        #print("Database connected successfully")


                        cur = conn.cursor() # creating a cursor

                        #ROUTE=var_validator(ROUTE):
                        try:
                            ROUTE
                        except NameError:
                            #print("well, it WASN'T defined after all!")
                            ROUTE='N/A'
                        #else:
                        #    print("sure, ROUTE was defined.")

                        try:
                            GUFI
                        except NameError:
                            #print("well, it WASN'T defined after all!")
                            GUFI='N/A'
                        #else:
                        #    print("sure, it was defined.")

                        try:
                            SCHE
                        except NameError:
                            #print("well, it WASN'T defined after all!")
                            SCHE='N/A'
                        #else:
                        #    print("sure, it was defined.")


                        try:
                            DEP
                        except NameError:
                            #print("well, it WASN'T defined after all!")
                            DEP='N/A'
                        #else:
                        #    print("sure, it was defined.")


                        try:
                            ARR
                        except NameError:
                            #print("well, it WASN'T defined after all!")
                            ARR='N/A'
                        #else:
                        #    print("sure, it was defined.")


                        try:
                            flightnumber
                        except NameError:
                            #print("well, it WASN'T defined after all!")
                            flightnumber='N/A'
                        #else:
                        #    print("sure, it was defined.")


                        try:
                            airline
                        except NameError:
                            #print("well, it WASN'T defined after all!")
                            airline='N/A'
                        #else:
                        #    print("sure, it was defined.")


                        print()
                        print('report')
                        print(ID,GUFI,flightnumber,SCHE,DEP,ARR,ROUTE,airline)
                        print('FILENAME: %s'%filename)
                        print()

                        try:
                            cur.execute("""
                                INSERT INTO aircraft (ID,gufi,aircraftid,igtd,departurePoint,arrivalPoint,route,Airline) VALUES
                                ('%s','%s','%s','%s','%s','%s','%s','%s')
                                    """%(ID,GUFI,ACID,SCHE,DEP,ARR,ROUTE,airline))
                            conn.commit()
                            conn.close()
                            del ROUTE
                            del ROUTE_tmp
                        except:
                             conn.close()
                        #    print('NO')

conn = ps.connect(database=DB_NAME,
						      user=DB_USER,
						      password=DB_PASS,
							    host=DB_HOST,
							    port=DB_PORT)
#print("Database connected successfully")

query="SELECT route,departurePoint,arrivalPoint FROM aircraft where route<>'N/A'"
df = pd.read_sql_query("%s"%query, con=conn)

print()
print(df)
print()

cursor = conn.cursor()

#query = "UPDATE table_name SET column_name = %s WHERE condition = %s"
#values = ("new value", "78914")
#cursor.execute(query, values)

cursor.execute(query)
results = cursor.fetchall()
#print(results)

for row in results:
        print(row)

print()

# Commit the transaction to save the changes
conn.commit()
#print("QUERY successfully")
print()

# Close the cursor and connection
cursor.close()
conn.close()

Streaming output truncated to the last 5000 lines.

report
454 KF45085600 AAL514 2023-09-22T14:12:00Z KDFW KJAC N/A AAL
FILENAME: /content/drive/MyDrive/dataswim/1695395334127


report
454 KF45085600 AAL514 2023-09-22T13:32:00Z KDFW KJAC N/A AAL
FILENAME: /content/drive/MyDrive/dataswim/1695395334127


report
456 KF45085600 AAL514 2023-09-22T13:32:00Z KDFW KJAC N/A AAL
FILENAME: /content/drive/MyDrive/dataswim/1695395334127


report
456 KF45085600 AAL514 2023-09-22T13:32:00Z KDFW KJAC N/A AAL
FILENAME: /content/drive/MyDrive/dataswim/1695395334127


report
461 KF45085600 AAL514 2023-09-22T13:32:00Z KDFW KJAC N/A AAL
FILENAME: /content/drive/MyDrive/dataswim/1695395334127


report
463 KF45085600 AAL514 2023-09-22T13:32:00Z KDFW KCLT N/A AAL
FILENAME: /content/drive/MyDrive/dataswim/1695395334127


report
467 KF45085600 AAL514 2023-09-22T13:32:00Z KDFW KCLT N/A AAL
FILENAME: /content/drive/MyDrive/dataswim/1695395334127


report
482 KF45085600 DAL809 2023-09-22T13:32:00Z KDFW KCLT N/A DA

<ipython-input-1-a7ebff0c17c0>:359: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("%s"%query, con=conn)



                                                 route departurepoint  \
0               KTOL./.SGF318046..CME.SAMMR3.KELP/1540           KTOL   
1                                           SCEL..SCCF           SCEL   
2             KMCO./.SAV001011..JURDI.DMSTR1.KRDU/1530           KMCO   
3    KSRQ.SRKUS4.CAMJO.Q99.TEEEM.Q109.DFENC..SAWED....           KSRQ   
4                   KATL.POUNC2.GRGIA..SJI.MNSTR1.KMSY            ATL   
..                                                 ...            ...   
703    KATL./.VXV200061..JONEN..MCGNS.SCRLT1.KCMH/1531           KATL   
704                KDEN./.IOW167047..JAKKS.JAKKS2.KIND           KDEN   
705        KBNA./.3958N/07454W..NEWES.ROBUC3.KBOS/1521           KBNA   
706           KIAD./.CRG133027..JIPOD.CUUDA2.KFLL/1529           KIAD   
707  KTEB..NEWEL.J60.DBL.J80.ILC..KNGRY.RAZRR5.KSJC...           KTEB   

    arrivalpoint  
0           KELP  
1           SCCF  
2           KRDU  
3           KJFK  
4            MSY  
..      

In [2]:
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASS = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

import os
import psycopg2 as ps
import pandas as pd

print()

conn = ps.connect(database=DB_NAME,
						      user=DB_USER,
						      password=DB_PASS,
							    host=DB_HOST,
							    port=DB_PORT)
print("Database connected successfully")

print()
query="SELECT route,departurePoint,arrivalPoint FROM aircraft where route<>'N/A'"
cursor = conn.cursor()
cursor.execute(query)
results = cursor.fetchall()
for row in results:
        print(row)

# Close the cursor and connection
cursor.close()
conn.close()


conn = ps.connect(database=DB_NAME,
						      user=DB_USER,
						      password=DB_PASS,
							    host=DB_HOST,
							    port=DB_PORT)
print("Database connected successfully")

print()

## PANDA
print('PANDA')
#query="SELECT count(*) FROM aircraft where route<>'N/A'"
query="SELECT route,departurePoint,arrivalPoint FROM aircraft where route<>'N/A'"
df = pd.read_sql_query("%s"%query, con=conn)
df

#conn.close()

#print(df[0])

# Close the cursor and connection
#cursor.close()
#conn.close()


Database connected successfully

('KTOL./.SGF318046..CME.SAMMR3.KELP/1540', 'KTOL', 'KELP')
('SCEL..SCCF', 'SCEL', 'SCCF')
('KMCO./.SAV001011..JURDI.DMSTR1.KRDU/1530', 'KMCO', 'KRDU')
('KSRQ.SRKUS4.CAMJO.Q99.TEEEM.Q109.DFENC..SAWED.Q97.DLAAY..RADDS..SIE.CAMRN4.KJFK/0218', 'KSRQ', 'KJFK')
('KATL.POUNC2.GRGIA..SJI.MNSTR1.KMSY', 'ATL', 'MSY')
('KAFW.NANDR2.ZEMMA.TSDEL3.KOKC/1525', 'KAFW', 'KOKC')
('PHNL./.3349N/12525W..GALIP..SERCO.RYDRR2.KLAX/1557', 'PHNL', 'KLAX')
('KBED..PEASE..ENE..KIZG/1529', 'KBED', 'KIZG')
('KEWR./.BAL291061..HVQ.Q68.RAMRD..SQS..YUYUN.WHINY4.KDFW', 'KEWR', 'KDFW')
('KPIR./.SNY141019..AALLE.AALLE3.KDEN/1507', 'KPIR', 'KDEN')
('KCLT./.CRG036042..URSUS.UL780.TASNO.UM221.NIBEO.UL341.ENARI.LENAR7.MKJS/1634', 'KCLT', 'MKJS')
('KMSP.RST1.ALO..KK54I..WHOLL.BRBBQ3.KMEM', 'MSP', 'MEM')
('PNE..HXD', 'PNE', 'HXD')
('KPHX./.FUZ240009..KAMEN.Q184.MERDN..ORRKK.GNDLF2.KATL/1625', 'KPHX', 'KATL')
('CYYC./.UBVAL.Q927.SEKOM..KEETA..DBS.NORDK6.KSLC', 'CYYC', 'KSLC')
('TNCA./.KNSLY331

<ipython-input-2-cd68260d07ea>:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("%s"%query, con=conn)


,route,departurepoint,arrivalpoint
0,KTOL./.SGF318046..CME.SAMMR3.KELP/1540,KTOL,KELP
1,SCEL..SCCF,SCEL,SCCF
2,KMCO./.SAV001011..JURDI.DMSTR1.KRDU/1530,KMCO,KRDU
3,KSRQ.SRKUS4.CAMJO.Q99.TEEEM.Q109.DFENC..SAWED....,KSRQ,KJFK
4,KATL.POUNC2.GRGIA..SJI.MNSTR1.KMSY,ATL,MSY
...,...,...,...
703,KATL./.VXV200061..JONEN..MCGNS.SCRLT1.KCMH/1531,KATL,KCMH
704,KDEN./.IOW167047..JAKKS.JAKKS2.KIND,KDEN,KIND
705,KBNA./.3958N/07454W..NEWES.ROBUC3.KBOS/1521,KBNA,KBOS
706,KIAD./.CRG133027..JIPOD.CUUDA2.KFLL/1529,KIAD,KFLL
